# 3D Contouring: Example

This is a simple notebook that shows the usage of the maximal empty spheres contouring as implemented in the  `LieConeSDFReconstruction` class in `lie_cone.py`.

In case the CGAL version was compiled as well (see instructions in its Readme.md), this notebook also includes an example how to use it.

A discrete set of signed distance function (SDF) values is sampled based on a ground truth mesh. 
The reconstructions for our empty spheres approach is shown along side ["Reach for the Arcs" (RFTA, Sellán et al. 2024)](https://dl.acm.org/doi/abs/10.1145/3641519.3657419).

Download and use the Armadillo mesh as ground truth. 
This is only an example, feel free to set `mesh_path` to a different `.obj` file.  

In [ ]:
%load_ext autoreload
%autoreload 2

import os
# Download the Armadillo from github
mesh_path = "armadillo.obj"
if not os.path.exists(mesh_path) and mesh_path == "armadillo.obj":
    import wget
    url = 'https://raw.githubusercontent.com/alecjacobson/common-3d-test-models/refs/heads/master/data/armadillo.obj'
    filename = wget.download(url)
    print()
    print(filename)
else:
    print("Found .obj file")

Sample SDF values on a regular grid. Note that the bounding box is chosen slightly larger than the object.

In [ ]:
import numpy as np

def sample_positions(n, random_sampling, sdf):
    if random_sampling:
        U = (np.random.rand(n*n*n, 3)-0.5)*2
    else:
        gx, gy, gz = np.meshgrid(np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1))
        U = np.vstack((gx.flatten(), gy.flatten(), gz.flatten())).T
    U_sdfvals = sdf(U)
    return U, U_sdfvals

import gpytoolbox as gpy

# Set up gt
V_gt, F_gt = gpy.read_mesh(mesh_path)
V_gt = gpy.normalize_points(V_gt)

s = 0.9 # 0.75
V_gt *= s/0.5

# Create and abstract SDF function that is the only connection to the shape
sdf = lambda x: gpy.signed_distance(x, V_gt, F_gt)[0]

In [ ]:
# number of samples per dimensions
n = 30

# Sample SDF values on a regular grid
U, U_sdfvals = sample_positions(n,False,sdf)

# all methods use the Screened Poisson Surface Reconstruction (sPSR) implementation as made available in the gpytoolbox  
# setting the screening weight a bit lower can help avoid screening artifacts
psr_screening_weight = 1.

### RFTA

In [ ]:
import time

# RFTA
strt = time.time()
R = gpy.reach_for_the_arcs(U, U_sdfvals,parallel=True,screening_weight=psr_screening_weight)
stp  = time.time()
print(f"RFTA (parallel): {stp-strt}s")

### Maximal Empty Spheres (Python)

In [ ]:
from lie_cone import LieConeSDFReconstruction

strt = time.time()
cone = LieConeSDFReconstruction(np.concatenate([U,U_sdfvals[:,None]],axis=1),
                                    filter_type=3,cut_bbx_factor=1.,filter_results=False,
                                   psr_screening_weight=psr_screening_weight)

stp  = time.time()
print(f"Empty spheres rec: {stp-strt}s")

R_lc = cone.V, cone.F

### Maximal Empty Spheres (cpp, experimental)

This part of the codes requires the binary for the cpp version. See the `cgal/` folder for instructions on how to compile it.

Note that this uses code that is still under development.

In [ ]:
LC_cgal_available=False
from cgal.EmptySpheresReconstruction import MESReconstruction
R_cgal = MESReconstruction(U,U_sdfvals,screening_weight=psr_screening_weight)
if R_cgal:
    LC_cgal_available=True

## Visualize the results using polyscope

In [ ]:
from matplotlib.colors import to_rgb
wong_colors = ["#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
method_colors = {"CP" : to_rgb(wong_colors[1]), 
                "RFTA": to_rgb(wong_colors[2]),
                "MC"  : to_rgb(wong_colors[3])}

In [ ]:
import polyscope as ps

ps.init()
ps.set_ground_plane_mode("none")

ps.register_surface_mesh("GT", V_gt, F_gt,color=to_rgb("tab:gray"))
ps.register_surface_mesh("RFTA", *R,color=method_colors["RFTA"])
ps.register_surface_mesh("LC", *R_lc,color=method_colors["CP"])

if LC_cgal_available:
    ps.register_surface_mesh("LC_cgal", *R_cgal, color=method_colors["CP"])

ps.show()